# Logging and Debugging

This lesson will deal on how to make sure that our code is doing what is supposed to be doing.

This is related to testing, but can be viewed as complementary to it

We will discuss:

* debugging
* assertions
* logging

and, given time:

* linters
* type checkers
* warnings


All these tools help us ensure that the program is doing what is supposed to be doing.

While tests check that the program logic is correct, these tools helps you check that the program is implementing the operations you think it's doing.

### the logic

most of these techniques are necessary to help you understand what you program is *actually* doing, instead of what you think it is supposed to be doing

## Debugging

A debugger is a program that attach itself to yours and monitor it state and allow you to control the execution in real time.

It doesn't sound that crazy for python, but consider that the original debuggers were doing it for C programs...

What we do when we execute our programs one line at the time from an editor like spyder is basically a manual for of a debugger

Python and IPython comes with a basic debugger for the command line, but most programs provide you with more advanced (and easy to use ones).

We will discuss the basic ones, but (as usual) just to explain the basic concepts that they use.

The basic operations can be divided in two main categories: 

* managing the execution
* examine and modify the program state

* **l**(list)	Lists the code at the current position
* **u**(p)	Walk up the call stack
* **d**(own)	Walk down the call stack
* **n**(ext)	Execute the next line (does not go down in new functions)
* **s**(tep)	Execute the next statement (goes down in new functions)
* **bt**	Print the call stack
* **a**	Print the local variables
* !command	Execute the given Python command (by opposition to pdb commands

* **h**(elp)	Show a list of commands, or find help on a specific command
* **q**(uit)	Quit the debugger and the program
* **c**(ontinue)	Quit the debugger, continue in the program
* `<Return>`	Repeat the previous command
* **p**(rint)	Print variables
* **s**(tep)	Step into a subroutine
* **r**(eturn)	Return out of a subroutine

In [1]:
%%file test.py

dati = [1, 2, 3, 4]

def my_function():
    for dato in dati:
        print(dati[dato])
        
my_function()

Overwriting test.py


In [23]:
%run test.py

2
3
4


IndexError: list index out of range

In [25]:
%run -d test.py

*** Blank or comment
*** Blank or comment
NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> /home/enrico/didattica/corso_programmazione_1819/programmingCourseDIFA/test.py(2)<module>()
      1 
----> 2 dati = [1, 2, 3, 4]
      3 
      4 def my_function():
      5     for dato in dati:

ipdb> n
> /home/enrico/didattica/corso_programmazione_1819/programmingCourseDIFA/test.py(4)<module>()
      2 dati = [1, 2, 3, 4]
      3 
----> 4 def my_function():
      5     for dato in dati:
      6         print(dati[dato])

ipdb> !print(dati)
[1, 2, 3, 4]
ipdb> !dati[-1] = 3
ipdb> !print(dati)
[1, 2, 3, 3]
ipdb> c
2
3
3
3


## Assertions

they are useful to express your expectations about the code

In [2]:
assert 1==0, "I was not expecting that"

AssertionError: I was not expecting that

but don't rely on it, as they can be removed from the execution by using the flag `-O` (Optimize) when calling the python interpreter.

Also, let's be honest, it does have a terrible syntax

We could use a better (albeit slower) assertion library, **grappa**, to help get better syntax and better error results.

In [2]:
from grappa import should, expect

In [3]:
try:
    'foo' | should.be.equal.to('bar')
except AssertionError as e:
    print(e)

Oops! Something went wrong!

  The following assertion was not satisfied
    subject "foo" should be equal to "bar"

  What we expected
    a value that is equal to "bar"

  What we got instead
    an value of type "str" with data "foo"

  Difference comparison
    > - foo
    > + bar

  Where
    File "<ipython-input-3-dd9644d0aacd>", line 2, in <module>

     1|   try:
     2| >     'foo' | should.be.equal.to('bar')
     3|   except AssertionError as e:
     4|       print(e)



In [28]:
with should({'foo': 'bar'}):
    should.be.a(dict)
    should.have.length(1)
    should.have.key('foo').that.should.be.equal.to('bar')

In [29]:
try:
    'hello' | should.be.empty
except AssertionError as e:
    print(e)

Oops! Something went wrong!

  The following assertion was not satisfied
    subject "hello" should be empty

  What we expected
    a value that is not "None" and its length is higher than zero

  What we got instead
    an object with type "str" which its length cannot be measured

  Information
    > An empty object can be "None", "0" or "len(x) == 0".
      Most objects in Python can be tested via "len(x)"
      such as str, list, tuple, dict, generator...
      as well as any object that implements "__len__()" method.
      => Reference: https://docs.python.org/3/library/functions.html#len

  Where
    File "<ipython-input-29-fac7d0be13c0>", line 2, in <module>

     1|   try:
     2| >     'hello' | should.be.empty
     3|   except AssertionError as e:
     4|       print(e)



In [33]:
data = 'hello'
try:
    expect(data).to.be.a('int')
except AssertionError as e:
    print(e)

Oops! Something went wrong!

  The following assertion was not satisfied
    subject "hello" expect to be a "<class 'int'>"

  What we expected
    an object that is a "<class 'int'>" type

  What we got instead
    an object of type "str" with value "hello"

  Difference comparison
    > - hello
    > + <class 'int'>

  Where
    File "<ipython-input-33-26e9dc31d426>", line 3, in <module>

     1|   data = 'hello'
     2|   try:
     3| >     expect(data).to.be.a('int')
     4|   except AssertionError as e:
     5|       print(e)



## Logging

When you execute your code and print the internal state of the program to check that is working properly, that is a rudimentary form of logging.

Printing the state of your program works fine as long as your program is simple and the amount of state is small.

For anything more complicated, you need to use a logging system.

The basic idea of a logging system is to standardize how and what gets written on a file

In [67]:
%%file test.py

from eliot import start_action, to_file, Message
to_file(open("test.log", "w"))

def myfunction(value):
    with start_action(action_type='myfunction', value=value):
        return 1/value

for number in [4, 1, 0, 2, 4]:
    with start_action(action_type="start evaluation", number=number):
        point = number *2
        total = sum(i for i in range(point))
        with start_action(action_type="inside evaluation", total=total):
            result = myfunction(total)


Overwriting test.py


In [68]:
!rm test.log
%run test.py

ZeroDivisionError: division by zero

In [56]:
!eliot-tree test.log

bc135185-26bb-44f4-adbf-93a762eca2f3
└── start evaluation/1 ⇒ started 2018-09-03 09:03:04 ⧖ 0.003s
    ├── number: 4
    ├── inside evaluation/2/1 ⇒ started 2018-09-03 09:03:04 ⧖ 0.002s
    │   ├── total: 28
    │   ├── myfunction/2/2/1 ⇒ started 2018-09-03 09:03:04 ⧖ 0.001s
    │   │   ├── value: 28
    │   │   └── myfunction/2/2/2 ⇒ succeeded 2018-09-03 09:03:04
    │   └── inside evaluation/2/3 ⇒ succeeded 2018-09-03 09:03:04
    └── start evaluation/3 ⇒ succeeded 2018-09-03 09:03:04

75834879-b121-40f8-97ef-04e27bbc6d61
└── start evaluation/1 ⇒ started 2018-09-03 09:03:04 ⧖ 0.002s
    ├── number: 1
    ├── inside evaluation/2/1 ⇒ started 2018-09-03 09:03:04 ⧖ 0.001s
    │   ├── total: 1
    │   ├── myfunction/2/2/1 ⇒ started 2018-09-03 09:03:04 ⧖ 0.000s
    │   │   ├── value: 1
    │   │   └── myfunction/2/2/2 ⇒ succeeded 2018-09-03 09:03:04
    │   └── inside evaluation/2/3 ⇒ succeeded 2018-09-03 09:03:04
    └── start evaluation/3 ⇒ succeeded 2018-09-03 09:03:04

e9c68561-2e34-45

In [51]:
!head test.log

{"action_type": "start evaluation", "task_level": [1], "timestamp": 1535965154.3335845, "task_uuid": "409f42e6-0c10-4f59-b882-e4fb7f112241", "action_status": "started", "number": 4}
{"action_type": "inside evaluation", "task_level": [2, 1], "timestamp": 1535965154.333918, "task_uuid": "409f42e6-0c10-4f59-b882-e4fb7f112241", "action_status": "started", "total": 28}
{"task_uuid": "409f42e6-0c10-4f59-b882-e4fb7f112241", "timestamp": 1535965154.3341036, "action_type": "inside evaluation", "task_level": [2, 2], "action_status": "succeeded"}
{"task_uuid": "409f42e6-0c10-4f59-b882-e4fb7f112241", "timestamp": 1535965154.3342803, "action_type": "start evaluation", "task_level": [3], "action_status": "succeeded"}
{"action_type": "start evaluation", "task_level": [1], "timestamp": 1535965154.334487, "task_uuid": "3d429872-a765-4c18-9559-cc41c68be49f", "action_status": "started", "number": 1}
{"action_type": "inside evaluation", "task_level": [2, 1], "timestamp": 1535965154.3346956, "task_uuid": "

What if you want to see the results in real time?

A better solution is to periodically refresh the result of `eliot-tree`, piping the tail of the log file in it:

    tail -f test.log | eliot-tree
    
for example, if we make our program be slower (simulating a slow computation)

The only limitations are:

* you have to interrupt the tail process manually
* actions are written only when completed, so if you have very high level actions this will not print until they are done

In [124]:
%%file test.py

import time
from eliot import start_action, to_file, Message, start_task
to_file(open("test.log", "w"))

def myfunction(value):
    with start_action(action_type='myfunction', value=value):
        time.sleep(5)
        result = 1/value
        Message.log(result=result)
        return 1/value

for number in [4, 1, 3, 5, 0, 2, 4]:
    with start_action(action_type="start evaluation", number=number):
        point = number *2
        total = sum(i for i in range(point))
        with start_action(action_type="inside evaluation", total=total):
            result = myfunction(total)

Overwriting test.py


In [125]:
!rm test.log
%run test.py

ZeroDivisionError: division by zero

Otherwise, if you are used to live in the matrix, you can watch the log file directly with:

    tail -f test.log

## Warning systems

Warnings are a way to comunicate directly with the user and let them know that there is something fishy going on.

While logging is something that is run consistently, warnings should appear only in some specific situations.

A tipical case is to inform your user that one of the functions that is being used is going to be removed from the next version of your library and that they should use something different

In [85]:
import warnings
warnings.warn("this is an old script, use a new one!")


/home/enrico/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: this is an old script, use a new one!
  


if you want to raise an error when you catch a warning, you can use a specific context manager

In [86]:
with warnings.catch_warnings():
        warnings.simplefilter('error', category=Warning)
        warnings.warn("there is a problem!")

UserWarning: there is a problem!

on the opposite, you might want to silence certain warnings as you know what you're doing.

SPOILER: you probably should not use this code!

In [88]:
with warnings.catch_warnings():
        warnings.simplefilter('ignore', category=Warning)
        warnings.warn("there is a problem!")

## Linters

Linters are programs that check your code for possible mistakes and errors before the execution.

there are several of them, with various level of informations that they can gather.

Most editors can be configured to run them in the background and show the resulting informations directly in the editor window.

examples of things that they will catch are:

* variable definited but not used
* overloading of existing functions
* syntax errors

and so on.

They are extremely useful when working with a dynamic language such as python, as they provide some functionalities of the traditional compilers

some of these linters are:

* pylint
* pycodestyle (previously called pep8)
* pyflakes
* flake8

In [93]:
%%file test.py

def eleva(n):
    return n**2

dati = [1, 2, 3, 4]

for dato in dati:
    print(eleva(dati[dato]))
    
print(data)

Overwriting test.py


In [94]:
!pylint test.py

************* Module test
test.py:9:0: C0303: Trailing whitespace (trailing-whitespace)
test.py:10:0: C0304: Final newline missing (missing-final-newline)
test.py:1:0: C0111: Missing module docstring (missing-docstring)
test.py:2:0: C0103: Argument name "n" doesn't conform to snake_case naming style (invalid-name)
test.py:2:0: C0111: Missing function docstring (missing-docstring)
test.py:5:0: C0103: Constant name "dati" doesn't conform to UPPER_CASE naming style (invalid-name)
test.py:10:6: E0602: Undefined variable 'data' (undefined-variable)

-------------------------------------------------------------------
Your code has been rated at -8.33/10 (previous run: 0.00/10, -8.33)



obviously not all suggestions are equally relevant, but they can spot several **code smells** before you run a long simulation

## Static type checking

A specific type of linter are the static type checkers, that are made possible by new syntax from python 3.

I will not get into the details of it, just the general idea.

Python is a dynamic language, but is still strongly typed.
One just don't have to declare the typed beforehand.

What they introduced is the possibility to annotate the code to express expectations over the type of variables, arguments and functions.
This does not have any effect on running the code, but allow type checkers (the most famous one is `mypy`) to assess if the code is correct

for example, the following code have a lot of issues, but they migh not be immediately apparent just looking at it.

If we don't put any typing information, mypy doesn't complain and doesn't do anything

In [95]:
%%file test.py

def eleva(n):
    return n.upper()

dati = [1, 2, 3, 4]

for dato in dati:
    print(eleva(dati[dato]))

Overwriting test.py


In [97]:
!mypy test.py

I can start introducing types informations and will find progressively more possible mistakes.

types are specified with a colon (`:`) after the argument or variable, followed by the type object that it should have.

for the return type, one can use an arrow `->`

In [113]:
%%file test.py

def eleva(n: str) -> str:
    return n.upper()

dati = [1, 2, 3, 4]

for dato in dati:
    print(1+eleva(dati[dato]))

Overwriting test.py


In [114]:
!mypy test.py

test.py:8: error: Unsupported operand types for + ("int" and "str")
test.py:8: error: Argument 1 to "eleva" has incompatible type "int"; expected "str"


I can specify more complex types using the library `typing`, that allows to specify very complex structures

In [115]:
%%file test.py

from typing import List

def eleva(n: str):
    return n.upper()

dati: List[str] = [1, 2, 3, 4]

for dato in dati:
    print(eleva(dati[dato]))

Overwriting test.py


In [109]:
!mypy test.py

test.py:7: error: List item 0 has incompatible type "int"; expected "str"
test.py:7: error: List item 1 has incompatible type "int"; expected "str"
test.py:7: error: List item 2 has incompatible type "int"; expected "str"
test.py:7: error: List item 3 has incompatible type "int"; expected "str"
test.py:10: error: No overload variant of "__getitem__" of "list" matches argument type "str"
test.py:10: note: Possible overload variants:
test.py:10: note:     def __getitem__(self, int) -> str
test.py:10: note:     def __getitem__(self, slice) -> List[str]


this allows me to annotate only the code I **care about** without having to deal with typing everywhere

you can find more info about tye checking and how to use it on this blog posts:
  
#### a general overview of the type systems
https://www.bernat.tech/the-state-of-type-hints-in-python/

#### an in-depth discussion of how the type system works and how to use it
* [S01E01](https://blog.daftcode.pl/first-steps-with-python-type-system-30e4296722af)
* [S01E02](https://blog.daftcode.pl/next-steps-with-python-type-system-efc4df5251c9)
* [S02E01](https://blog.daftcode.pl/csi-python-type-system-episode-1-1c2ee1f8047c)
* [S02E02](https://blog.daftcode.pl/csi-python-type-system-episode-2-baf5168038c0)
* [S02E03](https://blog.daftcode.pl/covariance-contravariance-and-invariance-the-ultimate-python-guide-8fabc0c24278)